In [2]:
%load_ext autoreload
%autoreload 2
from config import username, password
from modules.login_module import logIn, launch_to_homescreen, create_directory
from modules.download_files_module import request_report_process, download_loop_missing, download_process,  move_files_over, unzip_xlsx_file, unzip_files_in_same_dir, move_xlsx_files

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
import time
from datetime import datetime
today_date = datetime.now()
formatted_month_day = today_date.strftime("%m_%d")

logging.basicConfig(filename='ELPAC_SBAC_results.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)
logging.info('\n\n-------------ELPAC_SBAC_results new instance log')

#create file_download dir, and establish download directory
create_directory('file_downloads')
create_directory(f'file_downloads\\sbac\\{formatted_month_day}')
create_directory(f'file_downloads\\elpac\\{formatted_month_day}')

# Set up Chrome options
download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day}'
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_directory,
         'profile.default_content_setting_values.automatic_downloads': 1,
         'profile.content_settings.exceptions.automatic_downloads.*.setting': 1}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

logIn(username, password, driver)
launch_to_homescreen(driver)

coord_list = ['LEA CAASPP Coordinator at Alain Leroy Locke College Preparatory Academy',
 'LEA CAASPP Coordinator at Animo City of Champions Charter High',
 'LEA CAASPP Coordinator at Animo Compton Charter',
 'LEA CAASPP Coordinator at Animo Ellen Ochoa Charter Middle',
 'LEA CAASPP Coordinator at Animo Florence-Firestone Charter Middle',
 'LEA CAASPP Coordinator at Animo Inglewood Charter High',
 'LEA CAASPP Coordinator at Animo Jackie Robinson High',
 'LEA CAASPP Coordinator at Animo James B. Taylor Charter Middle',
 'LEA CAASPP Coordinator at Animo Jefferson Charter Middle',
 'LEA CAASPP Coordinator at Animo Leadership High',
 'LEA CAASPP Coordinator at Animo Legacy Charter Middle',
 'LEA CAASPP Coordinator at Animo Mae Jemison Charter Middle',
 'LEA CAASPP Coordinator at Animo Pat Brown',
 'LEA CAASPP Coordinator at Animo Ralph Bunche Charter High',
 'LEA CAASPP Coordinator at Animo South Los Angeles Charter',
 'LEA CAASPP Coordinator at Animo Venice Charter High',
 'LEA CAASPP Coordinator at Animo Watts College Preparatory Academy',
 'LEA CAASPP Coordinator at Oscar De La Hoya Animo Charter High',
 'LEA ELPAC Coordinator at Alain Leroy Locke College Preparatory Academy',
 'LEA ELPAC Coordinator at Animo City of Champions Charter High',
 'LEA ELPAC Coordinator at Animo Compton Charter',
 'LEA ELPAC Coordinator at Animo Ellen Ochoa Charter Middle',
 'LEA ELPAC Coordinator at Animo Florence-Firestone Charter Middle',
 'LEA ELPAC Coordinator at Animo Inglewood Charter High',
 'LEA ELPAC Coordinator at Animo Jackie Robinson High',
 'LEA ELPAC Coordinator at Animo James B. Taylor Charter Middle',
 'LEA ELPAC Coordinator at Animo Jefferson Charter Middle',
 'LEA ELPAC Coordinator at Animo Leadership High',
 'LEA ELPAC Coordinator at Animo Legacy Charter Middle',
 'LEA ELPAC Coordinator at Animo Mae Jemison Charter Middle',
 'LEA ELPAC Coordinator at Animo Pat Brown',
 'LEA ELPAC Coordinator at Animo Ralph Bunche Charter High',
 'LEA ELPAC Coordinator at Animo South Los Angeles Charter',
 'LEA ELPAC Coordinator at Animo Venice Charter High',
 'LEA ELPAC Coordinator at Animo Watts College Preparatory Academy',
 'LEA ELPAC Coordinator at Oscar De La Hoya Animo Charter High']

elpac_coordinators = [coord for coord in coord_list if 'ELPAC' in coord]
caaspp_coordinators = [coord for coord in coord_list if 'CAASPP' in coord]

#This exists when passing names into the requested reports, as a subset. Change list into a set to only retain unique schools
school_report_names = [entry.split(' at ', 1)[1] for entry in elpac_coordinators]
school_report_names = list(set(school_report_names))

# ---------------------------------------SBAC Files Request and Download

# Call the function, school report names variable is called for just school name
request_report_process(driver, 'SBAC', 'CAASPP_Student_Score_Data_Extract_Report', caaspp_coordinators)
download_process(school_report_names, '2023 CAASPP Student Score Data File By Enrolled LEA', driver) 

#This is here three times to see if anything got skipped the first time. Initial dir is set at ELPAC only to move the files over to SBAC dir
#Will run 5 times

time.sleep(10) #implemented to give time for files to download
download_loop_missing(f'elpac\\{formatted_month_day}', '2023 CAASPP Student Score Data File By Enrolled LEA', driver)

#This moves the files from ELPAC  timestamp dir to SBAC timestamp dir. 
#This is because the download dir cannot be changed in Selenium
move_files_over()

# --------------------------------------------ELPAC Files Request and Download

driver.switch_to.default_content()
request_report_process(driver, 'ELPAC', 'Student_Results_Report_Student_Score_Data_Extract', elpac_coordinators)
download_process(school_report_names, '2023 Summative ELPAC and Summative Alternate ELPAC Student Score Data File By Enrolled LEA', driver) 

time.sleep(10) #implemented to give time for files to download
#This is here three times to see if anything got skipped the first time. 
#Dir remains ELPAC for constant download directory
download_loop_missing(f'elpac\\{formatted_month_day}', '2023 Summative ELPAC and Summative Alternate ELPAC Student Score Data File By Enrolled LEA', driver)

#Takes 14 mins to run up to this point
# -----------------------------------------Unzip the Files and Move them to the P-Drive in this location 'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'
unzip_files_in_same_dir('elpac')
unzip_files_in_same_dir('sbac')

#Keeps raw zip files in the same dir. Only moves over xlsx files
move_xlsx_files('sbac')
move_xlsx_files('elpac')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Directory 'file_downloads' already exists.
Directory 'file_downloads\sbac\01_08' already exists.
Directory 'file_downloads\elpac\01_08' already exists.


C:\Users\SAMUEL~1.TAY\AppData\Local\Temp/ipykernel_13524/3624763137.py:39: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.199)
Stacktrace:
	GetHandleVerifier [0x01186EE3+174339]
	(No symbol) [0x010B0A51]
	(No symbol) [0x00DC6FF6]
	(No symbol) [0x00DAEFE7]
	(No symbol) [0x00E1B53B]
	(No symbol) [0x00E29E7B]
	(No symbol) [0x00E16DA6]
	(No symbol) [0x00DF1034]
	(No symbol) [0x00DF1F8D]
	GetHandleVerifier [0x01224B1C+820540]
	sqlite3_dbdata_init [0x012E53EE+653550]
	sqlite3_dbdata_init [0x012E4E09+652041]
	sqlite3_dbdata_init [0x012D97CC+605388]
	sqlite3_dbdata_init [0x012E5D9B+656027]
	(No symbol) [0x010BFE6C]
	(No symbol) [0x010B83B8]
	(No symbol) [0x010B84DD]
	(No symbol) [0x010A5818]
	BaseThreadInitThunk [0x756AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77637C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77637C3E+238]


# Solutions

In [ ]:
#Also another solution on the zip could be to only unzip the given excel file into the p-drive.

#Re-examine the logs and see where it can be more clear
#Re-assess logs again
#Lastly log the number of files that made it over to the p-drive

# Manual Checks

In [44]:
files = os.listdir(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\ELPAC_SBAC_results_selenium\file_downloads\sbac\01_05')
files = pd.DataFrame(files, columns = ['raw'])
files['dupe'] = files['raw'].str[:7]
files['dupe'] = files['dupe'].astype(str).str.lstrip('0')

school_ids = {'CHA': '136119',
                'ING': '1996586',
                'LEA': '1996313',
                'DLH': '101675',
                'SLA': '102434',
                'VEN': '106831',
                'BRW': '106849',
                'BUN': '111575',
                'ROB': '111583',
                'LCK': '118588',
                'JMS': '122481',
                'AEO': '123992',
                'LGC': '124016',
                'MAE': '129270',
                'FLO': '134023',
                'WAT': '111625',
                'JAM': '124008',
                'CMP': '137984'}


all_ = pd.DataFrame(list(school_ids.values()), columns = ['All Schools'])
pd.merge(files, all_, left_on='dupe', right_on='All Schools', how = 'outer', indicator=True)

,raw,dupe,All Schools,_merge
0,01016750000000.zip,101675,101675,both
1,01024340000000.zip,102434,102434,both
2,01068310000000.zip,106831,106831,both
3,01068490000000.zip,106849,106849,both
4,01115750000000.zip,111575,111575,both
5,01115830000000.zip,111583,111583,both
6,01116250000000.zip,111625,111625,both
7,01185880000000.zip,118588,118588,both
8,01224810000000.zip,122481,122481,both
9,01239920000000.zip,123992,123992,both
